In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from src.utils import prepare_df, group_cases_by_trace

In [11]:
# load your DataFrame and softmax list
result = prepare_df('50salads')
if len(result) == 2:
    df, softmax_lst = result
else:
    df, softmax_lst, _ = result

# group by trace and inspect
trace_groups = group_cases_by_trace(df)
trace_groups

,case_list,trace_length
0,"[0, 1, 2, 3]",5687
1,"[32, 33, 34, 35]",6186
2,"[36, 37, 38, 39]",5840
3,"[28, 29, 30, 31]",5261
4,"[4, 5, 6, 7]",6208
5,"[16, 17, 18, 19]",6293
6,"[24, 25, 26, 27]",6046
7,"[8, 9, 10, 11]",6584
8,"[12, 13, 14, 15]",5558
9,"[20, 21, 22, 23]",5792


In [ ]:
from src.data_processing import sample_and_write_collapsed_traces

# Sample using the same logic as incremental_softmax_recovery train split
sample_and_write_collapsed_traces(
    df, 
    n=10, 
    output_file_path='data/sampled_traces.txt', 
    random_seed=42,
    ensure_train_variant_diversity=True
)

Sampled 10 trace IDs: ['12', '32', '18', '1', '9', '37', '20', '4', '31', '24']


In [ ]:
# Verification: Compare selected cases with split_train_test
from src.data_processing import split_train_test

# Get train cases using split_train_test (same logic as incremental_softmax_recovery)
train_df, _ = split_train_test(
    df,
    n_train_traces=10,
    n_test_traces=None,
    train_cases=None,
    test_cases=None,
    ensure_train_variant_diversity=True,
    allow_train_cases_in_test=False,
    random_seed=42
)

train_case_ids_from_split = sorted(train_df['case:concept:name'].unique().tolist())
print(f"Train cases from split_train_test: {train_case_ids_from_split}")

# Call sample_and_write_collapsed_traces and get case IDs from the written file
sample_and_write_collapsed_traces(
    df,
    n=10,
    output_file_path='data/sampled_traces_verify.txt',
    random_seed=42,
    ensure_train_variant_diversity=True,
    verbose=True
)

# Read back the case IDs from the filtered dataframe logic
# (We'll simulate what the function does internally)
from src.data_processing import _get_derived_seed, _select_diverse_cases

train_seed = _get_derived_seed(42, "train")
sampled_case_ids = _select_diverse_cases(df, 10, train_seed)
sampled_case_ids_sorted = sorted(sampled_case_ids)
print(f"\nSampled cases from sample_and_write_collapsed_traces: {sampled_case_ids_sorted}")

# Compare
if train_case_ids_from_split == sampled_case_ids_sorted:
    print("\n✓ SUCCESS: Both functions selected the same cases!")
else:
    print(f"\n✗ MISMATCH: Cases differ")
    print(f"  Only in split_train_test: {set(train_case_ids_from_split) - set(sampled_case_ids_sorted)}")
    print(f"  Only in sample_and_write: {set(sampled_case_ids_sorted) - set(train_case_ids_from_split)}")


In [ ]:
"""
This cell demonstrates the usage of sample_and_write_collapsed_traces with specific parameters.
It samples 20 training cases using seed 123 and diversity selection, but overrides with 
the provided train_cases=['5', '10', '15'] (which takes precedence over n and diversity).
Writes the collapsed traces to 'data/my_traces.txt' without verbose output.
"""


sample_and_write_collapsed_traces(
    df=df,
    n=20,
    output_file_path='data/my_traces.txt',
    case_column='case:concept:name',
    activity_column='concept:name',
    random_seed=123,
    ensure_train_variant_diversity=True,
    train_cases=['5', '10', '15'],  # If provided, overrides n and diversity
    verbose=False
)